In [1]:
import os
import warnings
import copy
import pickle
import json
import numpy as np
import healpy as hp
import matplotlib.pylab as plt

import pandas as pd
import hvplot.pandas
import panel as pn
import hvplot

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u

from lsst_efd_client import EfdClient
from lsst.resources import ResourcePath

from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler.features import Conditions
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site


os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu"
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
os.environ["AWS_PROFILE"] = "default"

bucket = "s3://rubin:"

In [2]:
# If you want the range of times for a given dayobs
DAYOBS = '2024-05-06'
site = Site('LSST')
almanac = Almanac()
night_events = almanac.get_sunset_info(evening_date=DAYOBS, longitude=site.longitude_rad)
sunset = Time(night_events['sunset'], format='mjd', scale='utc')
sunrise = Time(night_events['sunrise'], format='mjd', scale='utc')
survey_length = sunrise.mjd - sunset.mjd
sunset.iso, sunrise.iso, survey_length

('2024-05-06 21:58:47.804', '2024-05-07 11:20:39.747', 0.5568511909805238)

In [3]:
# Connect to the EFD
efd_client = EfdClient('usdf_efd')
# efd_client = EfdClient('summit_efd') - at the summit

In [4]:
# Get list of scheduler snapshots from this dayobs
topic = 'lsst.sal.Scheduler.logevent_largeFileObjectAvailable'
fields = await efd_client.get_fields(topic)

df = await efd_client.select_time_series(topic, fields, sunset, sunrise, index=2)
print(len(df))
df['url']

19


2024-05-06 23:14:19.250380+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-06 23:25:41.764523+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-06 23:34:03.061203+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-06 23:42:43.301221+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-06 23:49:46.921542+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-06 23:58:28.060083+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-07 00:05:32.846180+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-07 00:12:49.374709+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-07 00:15:25.518930+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-07 00:20:54.769592+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-07 00:24:17.833505+00:00    https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2024-05-07 00:27:40.912991+00:00    https://s3.cp.lsst


Ok - let's try and match one of the snapshots from the dashboard so we can investigate times, whether targets actually acquired, etc.

Snapshot dashboard URL is at 
https://usdf-rsp-dev.slac.stanford.edu/schedview-snapshot/dashboard

Put in "Snapshot time" as "2024-05-07 15:58:29"

Pick a snapshot .. I picked one from the middle, to match above -- 

In [5]:
i = 9
print(df.index[i])
print(df['url'].iloc[i])

2024-05-07 00:20:54.769592+00:00
https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:2/Scheduler:2/2024/05/06/Scheduler:2_Scheduler:2_2024-05-07T00:21:30.395.p


In [6]:
url = df['url'].iloc[i].replace('https://s3.cp.lsst.org/', bucket)
# url = df['url'].iloc[i]  # -- at summit
uri = ResourcePath(url)
result = uri.read()

sched, conditions = pickle.loads(result)
assert isinstance(sched, CoreScheduler)
assert isinstance(conditions, Conditions)

conditions_time = Time(conditions.mjd, format='mjd', scale='utc')
sched_time = Time(sched.conditions.mjd, format='mjd', scale='utc')

print(conditions_time.iso)
print(sched_time.iso)

2024-05-07 00:26:16.000
2024-05-06 23:14:17.000


In [7]:
# These times are seriously different! 
(conditions_time - sched_time).sec / 60  # minutes

71.98333333362825

In [8]:
# What should have "won" from the scheduler snapshot?

#winner = sched.survey_lists[sched.survey_index[0]][sched.survey_index[1]]
sched.survey_index

# ok - None, None means the scheduler was not called yet. Which conditions should I use to find the winning observation??

[None, None]

In [9]:
# So now let's see what showed up in the target  ... which time?

time = conditions_time
# Add some wiggle room for the event time being recorded to database .. 
delta = TimeDelta(600, format='sec')

time, delta

(<Time object: scale='utc' format='mjd' value=60437.01824074074>,
 <TimeDelta object: scale='None' format='sec' value=600.0>)

In [10]:
# Get the topic and field information for the target events
topic = 'lsst.sal.Scheduler.logevent_target'
# fields = await efd_client.get_fields(topic) 
fields = ['note', 'ra', 'decl', 'skyAngle', 'filter', 'slewTime', 'exposureTimes0', 'exposureTimes1', 
          'airmass', 'skyBrightness', 'requestMjd', 'requestTime', 'targetId']

targets = await efd_client.select_time_series(topic, fields, time-delta, time, index=2)
targets

,note,ra,decl,skyAngle,filter,slewTime,exposureTimes0,exposureTimes1,airmass,skyBrightness,requestMjd,requestTime,targetId
2024-05-07 00:20:52.142884+00:00,AUXTEL_PHOTO_IMAGING:Photo0800-1_013,122.502619,-36.170921,5.301014,g,3.130053,120,0,1.178602,22.225946,0,0,0
2024-05-07 00:24:15.240543+00:00,AUXTEL_PHOTO_IMAGING:Photo0800-1_014,122.520953,-36.170921,6.083039,g,3.130042,120,0,1.193236,22.219376,0,0,0


In [11]:
# These do match up with the scheduler snapshot predicted "winners" ... 
# why are the scheduler conditions so old, why is the pickle conditions time later than the EFD time?
# could the scheduler be pickled after picking observations, so we record the "winner" in real time?

In [12]:
sched.update_conditions(conditions)
obs = sched.request_observation()

In [13]:
obs

array([(0, 2.13775407, -0.63130166, 0., 60437.041875, 120., 'g', 0.10317551, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'AUXTEL_PHOTO_IMAGING:Photo0800-1_012', '', 0, 0, 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0)],
      dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('note', '<U40'), ('target', '<U40'), ('field_id', '<i8'), ('survey_id', '<i8'), ('block_id', '<i8'), ('lmst', '<f8'), ('rotTelPos', '<f8'), ('rotTelPos_backup', '<f8'), ('moonAz', '<f8'), ('sunAz', '<f8'), 

In [15]:
print(sched.survey_index)
winner = sched.survey_lists[sched.survey_index[0]][sched.survey_index[1]]
winner

[1, 75]


<FieldSurvey survey_name='AUXTEL_PHOTO_IMAGING', RA=[2.13786835], dec=[-0.63128559] at 0x1a3d0af90>

In [ ]:
# Hmm - and this "winner" does not match the target requested. 